Import dependencies

In [64]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs

Load data

In [65]:
divorce_train = pd.read_csv("data/divorce.csv", sep=";")

print(divorce_train.head())

   Atr1  Atr2  Atr3  Atr4  Atr5  Atr6  Atr7  Atr8  Atr9  Atr10  ...  Atr46  \
0     2     2     4     1     0     0     0     0     0      0  ...      2   
1     4     4     4     4     4     0     0     4     4      4  ...      2   
2     2     2     2     2     1     3     2     1     1      2  ...      3   
3     3     2     3     2     3     3     3     3     3      3  ...      2   
4     2     2     1     1     1     1     0     0     0      0  ...      2   

   Atr47  Atr48  Atr49  Atr50  Atr51  Atr52  Atr53  Atr54  Class  
0      1      3      3      3      2      3      2      1      1  
1      2      3      4      4      4      4      2      2      1  
2      2      3      1      1      1      2      2      2      1  
3      2      3      3      3      3      2      2      2      1  
4      1      2      3      2      2      2      1      0      1  

[5 rows x 55 columns]


Prepare data

In [66]:
# Separate features from labels
divorce_features = divorce_train.copy()
divorce_labels = divorce_features.pop("Class")

# Take random indices for train/val/test
N_SAMPLES = 170
permutation = np.random.permutation(N_SAMPLES)
TRAIN_VAL_SEP = math.floor(0.7 * N_SAMPLES)
VAL_TEST_SEP = math.floor(0.85 * N_SAMPLES)
train_indices = permutation[:TRAIN_VAL_SEP]
validation_indices = permutation[TRAIN_VAL_SEP:VAL_TEST_SEP]
test_indices = permutation[VAL_TEST_SEP:]

# Take train/val/test form those random indices
xs_train = divorce_features.loc[train_indices]
ys_train = divorce_labels.loc[train_indices]

xs_val = divorce_features.loc[validation_indices]
ys_val = divorce_labels.loc[validation_indices]

xs_test = divorce_features.loc[test_indices]
ys_test = divorce_labels.loc[test_indices]

Encode data

In [67]:
def encode(v):
    return tf.one_hot(v, 2)

ys_train = encode(ys_train.to_numpy())
ys_val = encode(ys_val.to_numpy())
ys_test = encode(ys_test.to_numpy())

Compile model

In [68]:
divorce_model = tf.keras.Sequential()
divorce_model.add(tf.keras.Input(shape=54, dtype="float32"))
divorce_model.add(tf.keras.layers.Dropout(0.95))
divorce_model.add(tf.keras.layers.Dense(2, activation="ReLU"))
divorce_model.add(tf.keras.layers.Softmax())

divorce_model.compile(
    loss=tf.losses.BinaryCrossentropy(),
    optimizer=tf.optimizers.Adam(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

train model

In [69]:
EPOCHS = 1000
BATCH_SIZE = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)

divorce_model.fit(
    x=xs_train,
    y=ys_train,
    validation_data=(xs_val, ys_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    shuffle=True,
    callbacks=[callback]
)

Epoch 1/1000
118/118 [==============================] - 1s 3ms/step - loss: 2.4162 - categorical_accuracy: 0.6102 - val_loss: 0.6349 - val_categorical_accuracy: 0.6154
Epoch 2/1000
118/118 [==============================] - 0s 1ms/step - loss: 2.3432 - categorical_accuracy: 0.5932 - val_loss: 0.6122 - val_categorical_accuracy: 0.6154
Epoch 3/1000
118/118 [==============================] - 0s 1ms/step - loss: 1.6427 - categorical_accuracy: 0.6525 - val_loss: 0.6035 - val_categorical_accuracy: 0.6154
Epoch 4/1000
118/118 [==============================] - 0s 2ms/step - loss: 2.5209 - categorical_accuracy: 0.5593 - val_loss: 0.5722 - val_categorical_accuracy: 0.6538
Epoch 5/1000
118/118 [==============================] - 1s 5ms/step - loss: 2.3539 - categorical_accuracy: 0.5932 - val_loss: 0.5505 - val_categorical_accuracy: 0.6538
Epoch 6/1000
118/118 [==============================] - 0s 4ms/step - loss: 2.5047 - categorical_accuracy: 0.5847 - val_loss: 0.5182 - val_categorical_accuracy:

Evaluate model

In [70]:
print('\n Model evaluation test results')
divorce_model.evaluate(xs_test, ys_test)


 Model evaluation test results
1/1 [==============================] - 0s 122ms/step - loss: 0.3921 - categorical_accuracy: 1.0000


[0.3920785188674927, 1.0]

Sample prediction

In [71]:
print("Sample input (divorce questionnaire response)")
print(xs_test.to_numpy()[0])
print("\nSample divorce prediction [yes no]")
print(divorce_model.predict(xs_test)[0])
print("\nActual class (one-hot encoded)")
print(ys_test[0])

Sample input (divorce questionnaire response)
[2 0 2 0 0 0 0 0 0 2 0 0 2 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1
 0 1 1 0 2 1 0 0 3 2 2 1 1 1 1 2 1]

Sample divorce prediction [yes no]
[0.5 0.5]

Actual class (one-hot encoded)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)


Save model

In [72]:
divorce_model.save('nnModel/divorce_model.h5')

Convert to tfjs (optional but used in client-side deployment)

In [73]:
tfjs.converters.save_keras_model(divorce_model, 'tfjs')